## Setup Python Path

This cell adds the `scripts` directory to the Python path so we can import our custom Telegram loader module. This allows us to use the `TelegramMessageLoader` class we created.

In [1]:
import sys
import os
from pathlib import Path

# Add the scripts directory to the path so we can import our loader
sys.path.append(str(Path.cwd() / "scripts"))

## Environment Configuration

This cell loads the database credentials and configuration from your `.env` file. Make sure your `.env` file contains the correct PostgreSQL connection details for the `telegram_medical` database.

In [2]:
# If your .env is in the project root, this will load it
from dotenv import load_dotenv

load_dotenv(dotenv_path=Path.cwd() / ".env")

False

## Import Custom Loader

This cell dynamically imports our `TelegramMessageLoader` class from the `load_telegram_messages.py` script. This allows us to use the loader functionality directly in the notebook.

In [3]:
import importlib.util
from pathlib import Path

# If running from notebooks/ directory, go up one level to find scripts/
loader_path = Path.cwd().parent / "scripts" / "load_telegram_messages.py"
spec = importlib.util.spec_from_file_location("telegram_loader", loader_path)
telegram_loader = importlib.util.module_from_spec(spec)
spec.loader.exec_module(telegram_loader)

## Initialize Database Loader

This cell creates an instance of the `TelegramMessageLoader` class. The loader will:
- Connect to your PostgreSQL database
- Create the `raw.telegram_messages` table if it doesn't exist
- Set up the database schema and constraints

In [4]:
# Create an instance of the loader
loader = telegram_loader.TelegramMessageLoader()

2025-07-15 01:12:53,369 - ERROR - Failed to connect to database: No module named 'psycopg2'


ModuleNotFoundError: No module named 'psycopg2'